In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import bootstrap

In [2]:
# Test Dateset Preparation

test_data_folder = "./data/AReM/"
test_df_list = []


for dir_name in os.listdir(test_data_folder):
    activity_path = os.path.join(test_data_folder, dir_name)

    if os.path.isdir(activity_path):
        if dir_name in ["bending1", "bending2"]:
            selected_files = ["dataset1.csv", "dataset2.csv"]
        else:
            selected_files = ["dataset1.csv", "dataset2.csv", "dataset3.csv"]
            
        for file_name in selected_files:
            file_path = os.path.join(activity_path, file_name)

            if os.path.exists(file_path):
                df = pd.read_csv(file_path, delimiter=",", skiprows=4)
                df["Activity"] = dir_name
                test_df_list.append(df)


if test_df_list:
    test_df = pd.concat(test_df_list, ignore_index=False, axis=0)
    test_df.reset_index(drop=True, inplace=True)
    test_df = test_df.rename(columns = {'# Columns: time':'Time'})

test_df.head()

Time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  var_rss23  \
0     0      39.25       0.43      22.75       0.43      33.75        1.3   
1   250      39.25       0.43      23.00       0.00      33.00        0.0   
2   500      39.25       0.43      23.25       0.43      33.00        0.0   
3   750      39.50       0.50      23.00       0.71      33.00        0.0   
4  1000      39.50       0.50      24.00       0.00      33.00        0.0   

   Activity  
0  bending1  
1  bending1  
2  bending1  
3  bending1  
4  bending1

In [4]:
#training dataset

train_data_folder = "./data/AReM/"
train_df_list = []

for dir_name in os.listdir(train_data_folder):
    activity_path = os.path.join(train_data_folder, dir_name)

    if os.path.isdir(activity_path):
        
        if dir_name in ["bending1", "bending2"]:
            test_files = {"dataset1.csv", "dataset2.csv"}
        else:
            test_files = {"dataset1.csv", "dataset2.csv", "dataset3.csv"}

        train_files = [f for f in os.listdir(activity_path) if f.endswith('.csv') and f not in test_files]

        for file_name in train_files:
            file_path = os.path.join(activity_path, file_name)
            if os.path.exists(file_path):
                df = pd.read_csv(file_path, delimiter=",", skiprows=4)
                df["Activity"] = dir_name
                train_df_list.append(df)


if train_df_list:
    train_df = pd.concat(train_df_list, ignore_index=False, axis=0)
    train_df.reset_index(drop=True, inplace=True)
    train_df = train_df.rename(columns = {'# Columns: time':'Time'})


train_df.head()

Time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  var_rss23  \
0     0      42.00       0.00      18.50       0.50      12.00       0.00   
1   250      42.00       0.00      18.00       0.00      11.33       0.94   
2   500      42.75       0.43      16.75       1.79      18.25       0.43   
3   750      42.50       0.50      16.75       0.83      19.00       1.22   
4  1000      43.00       0.82      16.25       0.83      18.00       0.00   

   Activity  
0  bending1  
1  bending1  
2  bending1  
3  bending1  
4  bending1

In [5]:
data_folder = "./data/AReM/"
df_list = []

for dir_name in os.listdir(data_folder):
    activity_path = os.path.join(data_folder, dir_name)

    if os.path.isdir(activity_path):
        for csv_file in os.listdir(activity_path):
            if csv_file.endswith('.csv'):
                data_path_csv = os.path.join(activity_path, csv_file)
                df = pd.read_csv(data_path_csv, delimiter=",", skiprows=4)
                df_list.append(df)

len(df_list)

88

In [6]:
feature_list = []
for i, df in enumerate(df_list):  
    features = {"Instance": i + 1}
    
    for col_i, col in enumerate(df.columns[1:], start=1):
        features[f"min_{col_i}"] = np.min(df[col])
        features[f"max_{col_i}"] = np.max(df[col])
        features[f"mean_{col_i}"] = np.mean(df[col])
        features[f"median_{col_i}"] = np.median(df[col])
        features[f"STD_{col_i}"] = np.std(df[col])
        features[f"q1_{col_i}"] = np.percentile(df[col], 25)
        features[f"q3_{col_i}"] = np.percentile(df[col], 75)
    
    feature_list.append(features)

feature_df = pd.DataFrame(feature_list)
feature_df = feature_df.drop(columns=['Instance'])
print(feature_df.shape)
feature_df.head()

(88, 42)


min_1  max_1     mean_1  median_1     STD_1   q1_1   q3_1  min_2  max_2  \
0  36.25  48.00  43.969125     44.50  1.616677  43.31  44.67    0.0   1.50   
1  37.00  48.00  43.454958     43.25  1.384653  42.50  45.00    0.0   1.58   
2  33.00  47.75  42.179812     43.50  3.666840  39.15  45.00    0.0   3.00   
3  33.00  45.75  41.678063     41.75  2.241152  41.33  42.75    0.0   2.83   
4  37.25  45.00  40.624792     40.50  1.475428  39.25  42.00    0.0   1.30   

     mean_2  ...     STD_5     q1_5   q3_5  min_6  max_6    mean_6  median_6  \
0  0.413125  ...  3.314843  20.5000  23.75    0.0   2.96  0.555312      0.49   
1  0.378083  ...  2.486268  22.2500  24.00    0.0   5.26  0.679646      0.50   
2  0.696042  ...  3.845436  30.4575  36.33    0.0   2.18  0.613521      0.50   
3  0.535979  ...  2.408514  28.4575  31.25    0.0   1.79  0.383292      0.43   
4  0.358604  ...  2.186168  33.0000  36.00    0.0   1.92  0.570583      0.43   

      STD_6  q1_6  q3_6  
0  0.487318  0.00  0.83  
1  0.621885  0.43  0.87  
2  0.523771  0.00  1.00  
3  0.388759  0.00  0.50  
4  0.582308  0.00  1.30  

[5 rows x 42 columns]

In [7]:
std_feature_df = feature_df.describe()
std_feature_df = std_feature_df.loc[['std']].T
std_feature_df

std
min_1     9.569975
max_1     4.394362
mean_1    5.335718
median_1  5.440054
STD_1     1.770306
q1_1      6.153590
q3_1      5.138925
min_2     0.000000
max_2     5.062729
mean_2    1.574164
median_2  1.412244
STD_2     0.883184
q1_2      0.946386
q3_2      2.125266
min_3     2.956462
max_3     4.875137
mean_3    4.008380
median_3  4.036396
STD_3     0.945724
q1_3      4.220658
q3_3      4.171628
min_4     0.000000
max_4     2.183625
mean_4    1.166114
median_4  1.145586
STD_4     0.457764
q1_4      0.843620
q3_4      1.552504
min_5     6.124001
max_5     5.741238
mean_5    5.675593
median_5  5.813782
STD_5     1.023830
q1_5      6.096465
q3_5      5.531720
min_6     0.045838
max_6     2.518921
mean_6    1.154812
median_6  1.086474
STD_6     0.517078
q1_6      0.758584
q3_6      1.523599

In [8]:
n_trials = 1000
ci_dict = {}
for col in feature_df.columns:
    col_values = feature_df[col].dropna().values
    bootstrap_std = bootstrap((feature_df[col].values,), np.std, confidence_level=0.9, n_resamples=n_trials, method='percentile')
    ci_dict[col] = (bootstrap_std.confidence_interval.low, bootstrap_std.confidence_interval.high)

ci_df = pd.DataFrame(ci_dict, index=['Lower Bound', 'Upper Bound']).T
std_feature_df.join(ci_df)

std  Lower Bound  Upper Bound
min_1     9.569975     8.222084    10.710577
max_1     4.394362     3.325934     5.244509
mean_1    5.335718     4.674034     5.847397
median_1  5.440054     4.774010     6.034362
STD_1     1.770306     1.564938     1.936584
q1_1      6.153590     5.560690     6.612223
q3_1      5.138925     4.307651     5.827140
min_2     0.000000     0.000000     0.000000
max_2     5.062729     4.596498     5.370562
mean_2    1.574164     1.388451     1.699318
median_2  1.412244     1.230617     1.533792
STD_2     0.883184     0.799429     0.938749
q1_2      0.946386     0.820643     1.031625
q3_2      2.125266     1.882043     2.286730
min_3     2.956462     2.750977     3.089422
max_3     4.875137     4.203685     5.456175
mean_3    4.008380     3.391064     4.474769
median_3  4.036396     3.402587     4.498046
STD_3     0.945724     0.756920     1.117713
q1_3      4.220658     3.602486     4.686296
q3_3      4.171628     3.514609     4.694630
min_4     0.000000     0.000000     0.000000
max_4     2.183625     1.971282     2.343260
mean_4    1.166114     1.070403     1.213850
median_4  1.145586     1.051150     1.194557
STD_4     0.457764     0.418202     0.482573
q1_4      0.843620     0.766999     0.884818
q3_4      1.552504     1.431471     1.616655
min_5     6.124001     4.371912     7.419016
max_5     5.741238     4.722180     6.544704
mean_5    5.675593     4.354099     6.726125
median_5  5.813782     4.558862     6.925588
STD_5     1.023830     0.815930     1.205564
q1_5      6.096465     4.702213     7.150285
q3_5      5.531720     4.330812     6.511254
min_6     0.045838     0.000000     0.078029
max_6     2.518921     2.242167     2.740863
mean_6    1.154812     1.050840     1.204077
median_6  1.086474     0.992275     1.143533
STD_6     0.517078     0.474955     0.543365
q1_6      0.758584     0.687096     0.805500
q3_6      1.523599     1.402917     1.591457